In [ ]:
import urllib
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
import PIL.Image
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,5)
import numpy as np

In [ ]:
image_path = tf.keras.utils.get_file("cat.jpg","https://storage.googleapis.com/download.tensorflow.org/example_images/320px-Felis_catus-cat_on_snow.jpg")

In [ ]:
PIL.Image.open(image_path)

In [ ]:
# image to Tensor

In [ ]:
image_String = tf.io.read_file(image_path)

In [ ]:
image_String.shape

In [ ]:
image = tf.image.decode_jpeg(image_String,channels=3)

In [ ]:
image

In [ ]:
def visualize(orignal,augmented):
    fig = plt.figure()
    plt.subplot(1,2,1)
    plt.title('Original')
    plt.imshow(orignal)
    plt.subplot(1,2,2)
    plt.title('Augmented')
    plt.imshow(augmented)

In [ ]:
flipped = tf.image.flip_left_right(image)

In [ ]:
visualize(image,flipped)

In [ ]:
grayscaled = tf.image.rgb_to_grayscale(image)

In [ ]:
visualize(image,tf.squeeze(grayscaled))
plt.colorbar()

In [ ]:
saturated = tf.image.adjust_saturation(image,5)
visualize(image,saturated)

In [ ]:
bright = tf.image.adjust_brightness(image,0.4)
visualize(image,bright)

In [ ]:
rotated = tf.image.rot90(image)
visualize(image,rotated)

In [ ]:
cencropped = tf.image.central_crop(image,central_fraction=0.5)
visualize(image,cencropped)

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
# pip install tensorflow_datasets
import tensorflow_datasets as tfds


In [ ]:
dataset, info = tfds.load('mnist',as_supervised=True,with_info=True)

In [ ]:
train_dataset,test_dataset = dataset['train'],dataset['test']

In [ ]:
num_train_examples = info.splits['train'].num_examples

In [ ]:
num_train_examples

In [ ]:
info

In [ ]:
def convert(image,label):
    image = tf.image.convert_image_dtype(image,tf.float32)
    return image,label

In [ ]:
def augment(image,label):
    image,label=convert(image,label)
    image = tf.image.convert_image_dtype(image,tf.float32)
    image = tf.image.resize_with_crop_or_pad(image,34,34)
    image = tf.image.random_crop(image,size=[28,28,1])
    image = tf.image.random_brightness(image,max_delta=0.5)
    return image,label

In [ ]:
BATCH_SIZE = 64
NUM_EXAMPLES = 2048

In [ ]:
augmented_train_batch = ( train_dataset.take(NUM_EXAMPLES).cache().shuffle(num_train_examples//4).map(augment,num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE))

In [ ]:
augmented_train_batch

In [ ]:
non_augmented_train_batch = ( train_dataset.take(NUM_EXAMPLES).cache().shuffle(num_train_examples//4).map(convert,num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE))

In [ ]:
validation_batches = (test_dataset.map(convert,num_parallel_calls=AUTOTUNE).batch(2*BATCH_SIZE))

In [ ]:
def make_model():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28,28,1)),
        layers.Dense(4096,activation='relu'),
        layers.Dense(4096,activation='relu'),
        layers.Dense(10)
    ])
    model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    return model

In [ ]:
model_without_aug = make_model()

In [ ]:
no_aug_history = model_without_aug.fit(non_augmented_train_batch,epochs=50,validation_data=validation_batches)

In [ ]:
model_with_aug = make_model()

In [ ]:
aug_history = model_with_aug.fit(augmented_train_batch,epochs=50,validation_data=validation_batches)

In [ ]:
plt.plot(aug_history.history['accuracy'],label='Augmented')
plt.plot(no_aug_history.history['accuracy'],label='Non-Augmented')
plt.ylim([0.75,1])
plt.title('Accuracy Graph')
plt.ylabel('Accuracy')
plt.xlabel('Epoche')
plt.legend()
plt.show()

In [ ]:
plt.plot(aug_history.history['loss'],label='Augmented')
plt.plot(no_aug_history.history['loss'],label='Non-Augmented')
plt.ylim([0,1])
plt.title('Loss Graph')
plt.ylabel('Loss')
plt.xlabel('Epoche')
plt.legend()
plt.show()